In [1]:
from mae.encoder import *
from mae.decoder import *
from dataset import *
import gin

In [2]:
gin.parse_config_file("config/encoder_config1.gin")
encoder_config = EncoderConfig()

gin.parse_config_file("config/decoder_config1.gin")
decoder_config = DecoderConfig()
decoder_config.do_loss_calculation = True

In [3]:
decoder_config

DecoderConfig(image_size=128, in_proj_dim=256, hidden_size=256, intermediate_size=512, num_hidden_layers=4, num_attention_heads=2, num_channels=3, patch_size=8, norm_eps=1e-08, attention_dropout=0.0, do_loss_calculation=True, use_small_mlp=True, num_image_tokens=256, head_dim=128, patched_image_height=16, patched_image_width=16, rng_seed=42, rng_generator=<torch._C.Generator object at 0x000001D8CC2A7E90>)

In [4]:
encoder = EncoderModel(encoder_config)
decoder = DecoderModel(decoder_config)

In [5]:
# img, class_name = next(iter(train_dataloader))
# img.shape, class_name
import torch
import torchvision
# read image using torch 
img = torchvision.io.read_image("dataset/budgie/38.jpg")
img = img.to(torch.float32) / 255.0
img = img.unsqueeze(0)
img.shape

torch.Size([1, 3, 128, 128])

In [6]:
encoder_op, mask, ids_restore = encoder(img)
op, loss, mask, ids_restore = decoder((encoder_op, mask, ids_restore), img)

In [7]:
op.shape, mask.shape, ids_restore.shape, loss

(torch.Size([1, 3, 128, 128]),
 torch.Size([1, 256]),
 torch.Size([1, 256]),
 tensor(0.5284, grad_fn=<SqrtBackward0>))